# Visualizations

In this notebook, we'll create visualizations that will be used to best communicate the model process in the presentatino

In [2]:
## Import models relevant visualization models
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import geopandas as gpd